In [2]:
import typing

import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
import re

from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
nltk.download('wordnet')

from sklearn.metrics.pairwise import cosine_similarity
import config
import src
import requests
import tqdm
import json
import numpy as np
import logging
import torch

from transformers import AutoTokenizer, AutoModel

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sstolwijk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/sstolwijk/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/home/sstolwijk/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
CFG = config.Config()

In [4]:
dataset: pd.DataFrame = pd.read_parquet(f'{CFG.report_dir}/pubsphere.MXPembeds.parquet')
dataset

,StartDate,RecordedDate,IPAddress,Finished,Coder,ID,Mark_ID,Genre,topiccode,Platform,...,date_difference,commentCount_video,replyCount_comment,topic,subscribers,HATELIST_FOCUSED_DUMMY,Time_comment_year,Time_video_year,tfidf_embedding,embed_MXP
0,5/30/2021 13:03:17,5/30/2021 13:04:17,62.194.51.29,1,6,UgyPHwv8G0cDE6-wEgl4AaABAg.8_0ZjJKSJty8_0kXGkAd2U,119,0,0,1,...,NaN,NaN,NaN,None,NaN,0,2017,2017.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.40566757321357727, -0.032418690621852875, -..."
1,10/11/2021 10:34:05,10/11/2021 10:36:46,213.127.109.191,1,6,Ugx2WXq9UdV8mPPjejJ4AaABAg.8yHCKV0Boe58yYRxEQEF45,282,1,2,1,...,743.0,1748.0,NaN,economy,3630000.0,0,2019,2019.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.49942252039909363, -0.22240903973579407, -0..."
2,9/9/2021 18:49:48,9/9/2021 18:51:32,213.127.110.0,1,6,1110578710648890000,372,2,4,2,...,NaN,NaN,NaN,None,NaN,0,2019,NaN,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.6132397651672363, -0.3024018704891205, -0.6..."
3,6/6/2021 16:12:46,6/6/2021 16:16:16,213.127.76.145,1,6,UgwUPFScjJ0MCeaP2F54AaABAg.8lvp3fc9Euf8lvvgsUgEgV,769,0,0,1,...,NaN,NaN,NaN,None,NaN,0,2018,2018.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.27884048223495483, -0.3008716404438019, -0...."
4,6/13/2021 13:25:49,6/13/2021 13:27:28,213.127.82.232,1,6,UgwWKCWtSJdFvjGHvTp4AaABAg.8kUC5dGrQ2H8kUDRihE2f3,1206,0,0,1,...,NaN,NaN,NaN,None,NaN,0,2018,2018.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.5210463404655457, 0.03200243413448334, -0.3..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3857,8/19/2021 14:50:13,8/19/2021 14:54:28,62.194.51.29,1,6,1152219467579100000,10000695,0,4,2,...,NaN,NaN,NaN,None,NaN,0,2019,NaN,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.5265525579452515, -0.1172720193862915, -0.5..."
3858,8/19/2021 15:10:27,8/19/2021 15:12:21,62.194.51.29,1,6,1085362296472430000,10007008,1,4,2,...,NaN,NaN,NaN,None,NaN,0,2019,NaN,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.5607972741127014, -0.20688456296920776, -0...."
3859,10/6/2021 16:08:39,10/6/2021 16:10:42,213.127.113.113,1,6,UghFY3QJ6nmT_ngCoAEC.7-H0Z7--wxd8goqpaPs-bl,20000102,0,3,1,...,3803.0,4785.0,NaN,east,6740000.0,0,2018,2010.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.7534456253051758, -0.44059860706329346, -0...."
3860,10/15/2021 18:30:04,10/15/2021 18:35:40,213.127.109.191,1,6,UgyWabsmmnq3zam4DgZ4AaABAg,20000418,2,3,1,...,1531.0,2206.0,0.0,east,6800000.0,0,2018,2015.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.6457327604293823, -0.21125030517578125, -0...."


In [8]:
#first just try pre-processing and a simple tf-ivf:
lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words('english')

In [9]:
def preprocess(texts):
    x_train = []
    for sent in tqdm.tqdm(texts):
        sent = re.sub(r'@[^ ]+', '', sent)  #remove all usernames
        sent = re.sub(r'https?://[^ ]+', '', sent) #remove all hyperlinks
        sent = re.sub(r'#', '', sent) #remove all hashtags
        sent = re.sub(r'([A-Za-z])\1{2,}', r'\1', sent) #normalize language use by replacing duplicate letters by single letters
        sent = re.sub("[^a-zA-Z ]", "", sent) #remove all non-words
        sent = sent.lower().split()
        sent = [lemmatizer.lemmatize(word) for word in sent if word not in set(stop_words)]
        sent = ' '.join(sent)
        x_train.append(sent)
    return x_train

In [10]:
X = preprocess(dataset["commentText"])

100%|██████████| 3862/3862 [00:03<00:00, 1196.74it/s]


In [11]:
tfidf = TfidfVectorizer(max_features=5000, analyzer='word', ngram_range=(1,2), stop_words='english')
X_tfidf = tfidf.fit_transform(X).toarray()

In [12]:
X_tfidf[:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [13]:
dataset['tfidf_embedding'] = [torch.tensor(X_tfidf[i], dtype=torch.float32).flatten().tolist() for i in range(X_tfidf.shape[0])]


In [14]:
dataset['tfidf_embedding']

0       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
                              ...                        
3857    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3858    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3859    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3860    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3861    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: tfidf_embedding, Length: 3862, dtype: object

In [15]:
cosine_similarities = cosine_similarity(X_tfidf)
cosine_similarities

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.01644865, 0.10141343,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.08009685, 0.        ,
        0.        ],
       ...,
       [0.        , 0.01644865, 0.08009685, ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.10141343, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [16]:
num_documents = len(cosine_similarities)

# Initialize a list to store the most similar document pairs
top_similar_pairs = []

# Iterate through all document pairs
for i in range(num_documents):
    for j in range(i + 1, num_documents):
        similarity = cosine_similarities[i][j]
        
        # Check if the similarity is NaN or zero
        if not np.isnan(similarity) and similarity >= 1:
            pair = (i, j)
            top_similar_pairs.append((similarity, pair))
            
# Sort the list based on similarity in descending order
top_similar_pairs.sort(key=lambda x: x[0], reverse=True)
            


In [ ]:
top_similar_pairs

In [ ]:
result = [pair for value, pair in top_similar_pairs if value >= 1]
print(result)

In [ ]:
for first, second in result:
    print(dataset["commentText"][[first, second]])
    print()

In [19]:
print(X[246])
print(X[1179])
print(X[139])
print(X[1990])
print(X[21])
print(X[1856])
print(X[722])
print(X[739])
print(X[362])
print(X[736])

hell yeah fam
hell yeah
vote trump
vote trump
mika low iq
still low iq
good grief
opinionlol good grief
persinger apple orange
apple orange


In [31]:
grouped_data = dataset.groupby("Platform")
dist = torch.nn.PairwiseDistance()
resultsplatform: typing.Dict[typing.Tuple[str, str], float] = {}
for model_1, c_1 in tqdm.tqdm(grouped_data['tfidf_embedding'], total=grouped_data.ngroups):
    for model_2, c_2 in tqdm.tqdm(grouped_data['tfidf_embedding'], total=grouped_data.ngroups):

        if (
            (model_1, model_2) in resultsplatform.keys() or 
            (model_2, model_1) in resultsplatform.keys()
        ):
            continue

        res = sum([
            sum(dist(
                torch.tensor(np.array(v_1)), 
                torch.tensor(np.array(c_2.tolist()))
                )) / len(c_2)
            for v_1 in c_1
        ]) / len(c_1)

        resultsplatform[(model_1, model_2)] = res

        print(f'{model_1}:{model_2}:{res.item()}')

 50%|█████     | 1/2 [33:13<33:13, 1993.47s/it]

1:1:1.387083901878089



 50%|█████     | 1/2 [41:02<41:02, 2462.99s/it]

1:2:1.378470503605204



100%|██████████| 2/2 [42:52<00:00, 1286.27s/it]

2:2:1.3663818729263082


In [32]:
grouped_data = dataset.groupby("topiccode")
dist = torch.nn.PairwiseDistance()
resultstopic: typing.Dict[typing.Tuple[str, str], float] = {}
for model_1, c_1 in tqdm.tqdm(grouped_data['tfidf_embedding'], total=grouped_data.ngroups):
    for model_2, c_2 in tqdm.tqdm(grouped_data['tfidf_embedding'], total=grouped_data.ngroups):

        if (
            (model_1, model_2) in resultstopic.keys() or 
            (model_2, model_1) in resultstopic.keys()
        ):
            continue

        res = sum([
            sum(dist(
                torch.tensor(np.array(v_1)), 
                torch.tensor(np.array(c_2.tolist()))
                )) / len(c_2)
            for v_1 in c_1
        ]) / len(c_1)

        resultstopic[(model_1, model_2)] = res

        print(f'{model_1}:{model_2}:{res.item()}')

 20%|██        | 1/5 [01:36<06:27, 96.83s/it]

0:0:1.3645636310364535



 40%|████      | 2/5 [03:31<05:22, 107.47s/it]

0:1:1.3801746759070188



 60%|██████    | 3/5 [05:23<03:39, 109.52s/it]

...

3:3:1.390982498968042



 80%|████████  | 4/5 [28:50<06:28, 388.36s/it]

3:4:1.381690663108306



100%|██████████| 5/5 [30:39<00:00, 367.87s/it]

4:4:1.3663818729263082


In [33]:
grouped_data = dataset.groupby("Genre")
dist = torch.nn.PairwiseDistance()
resultsgenre: typing.Dict[typing.Tuple[str, str], float] = {}
for model_1, c_1 in tqdm.tqdm(grouped_data['tfidf_embedding'], total=grouped_data.ngroups):
    for model_2, c_2 in tqdm.tqdm(grouped_data['tfidf_embedding'], total=grouped_data.ngroups):

        if (
            (model_1, model_2) in resultsgenre.keys() or 
            (model_2, model_1) in resultsgenre.keys()
        ):
            continue

        res = sum([
            sum(dist(
                torch.tensor(np.array(v_1)), 
                torch.tensor(np.array(c_2.tolist()))
                )) / len(c_2)
            for v_1 in c_1
        ]) / len(c_1)

        resultsgenre[(model_1, model_2)] = res

        print(f'{model_1}:{model_2}:{res.item()}')

 33%|███▎      | 1/3 [04:55<09:51, 295.67s/it]

0:0:1.3860901042757274



 67%|██████▋   | 2/3 [10:25<05:15, 315.85s/it]

0:1:1.387775928468115



 33%|███▎      | 1/3 [15:47<31:35, 947.74s/it]

...

1:1:1.3867993339415101



 67%|██████▋   | 2/3 [27:58<13:40, 820.12s/it]

1:2:1.3827519017279932



100%|██████████| 3/3 [33:52<00:00, 677.35s/it]

2:2:1.3755636580085255


In [34]:
embed_MXBAI: typing.Dict[str, np.ndarray] = {}

In [76]:
dataset.columns


Index(['StartDate', 'RecordedDate', 'IPAddress', 'Finished', 'Coder', 'ID',
       'Mark_ID', 'Genre', 'topiccode', 'Platform', 'Anonymity',
       'Anonymity_9_TEXT', 'codable', 'Interaction', 'Acknowledgement',
       'TopicRelevance', 'Reasoning', 'BackgroundInfo', 'ExternalEvidence',
       'ExternalEvidence_1_TEXT', 'Opinion', 'disagreement',
       'Ideologicaldirection', 'Name_calling', 'Vulgarity',
       'Attack_reputation', 'Question_Intelligenc', 'All_caps_function',
       'Sarcasm_to_criticize', 'Individual_right', 'discrimination',
       'Invoke_violence', 'Tone', 'INTERACTIVITY_DUMMY', 'RATIONALITY_DUMMY',
       'HAS_OPINION_DUMMY', 'LIBERAL_NEUTRAL_CONSERVATIVE', 'LIBERAL_DUMMY',
       'CONSERVATIVE_DUMMY', 'NAMECALLING_DUMMY', 'VULGAR_DUMMY',
       'NAMECALLING_VULGAR_DUMMY', 'INCIVILITY_ORDINAL', 'INCIVILITY_DUMMY',
       'INTOLERANCE_DUMMY', 'filter_$', 'IMPOLITENESS_DUMMY', 'commentText',
       'showName', 'genre', 'Time_comment', 'likeCount_comment', 'entitie

In [71]:
context = 'social media replies to a news- or infotainment-post'
for index, row in tqdm.tqdm(dataset["commentText"].items(), total=len(dataset)):
    try: 
        embed = np.array(requests.post(
            'https://inf.cl.uni-trier.de/embed/',
            json={'prompt': 'You help me get embeddings for a sentence. I provide you with a context and a sentence and you reply only with that exact sentence. Context = ' + context + '; Sentence: ' + row}
            ).json()["response"])
    except Exception as _e:
        logging.warning(_e)
        embed = None
    
    embed_MXBAI[index] = embed

 56%|█████▌    | 2162/3862 [1:17:57<1:01:17,  2.16s/it]


In [77]:
print(embed_MXBAI[0])
print(embed_MXBAI[3])
print(embed_MXBAI[4])
print(embed_MXBAI[1800])

[ 0.40566757 -0.03241869 -0.53458202 ...  0.15711065 -0.08119941
 -0.00576604]
[ 0.27884048 -0.30087164 -0.48076391 ...  0.24177188 -0.36920482
  0.2004005 ]
[ 0.52104634  0.03200243 -0.37679625 ...  0.07765494 -0.12583201
  0.07684822]
[ 0.41127884 -0.00681883 -0.75118834 ... -0.06884278  0.1962322
  0.15082534]


In [75]:
dataset_w_embeds = dataset.join(pd.Series(embed_MXBAI, name="embed_MXBAI"))
dataset_w_embeds.to_parquet(f'{CFG.report_dir}/pubsphere.MXBAIembeds.parquet')
dataset_w_embeds.head()

,StartDate,RecordedDate,IPAddress,Finished,Coder,ID,Mark_ID,Genre,topiccode,Platform,...,date_difference,commentCount_video,replyCount_comment,topic,subscribers,HATELIST_FOCUSED_DUMMY,Time_comment_year,Time_video_year,tfidf_embedding,embed_MXP
0,5/30/2021 13:03:17,5/30/2021 13:04:17,62.194.51.29,1,6,UgyPHwv8G0cDE6-wEgl4AaABAg.8_0ZjJKSJty8_0kXGkAd2U,119,0,0,1,...,NaN,NaN,NaN,NaN,NaN,0,2017,2017.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.40566757321357727, -0.032418690621852875, -..."
1,10/11/2021 10:34:05,10/11/2021 10:36:46,213.127.109.191,1,6,Ugx2WXq9UdV8mPPjejJ4AaABAg.8yHCKV0Boe58yYRxEQEF45,282,1,2,1,...,743.0,1748.0,NaN,economy,3630000.0,0,2019,2019.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.49942252039909363, -0.22240903973579407, -0..."
2,9/9/2021 18:49:48,9/9/2021 18:51:32,213.127.110.0,1,6,1110578710648890000,372,2,4,2,...,NaN,NaN,NaN,NaN,NaN,0,2019,NaN,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.6132397651672363, -0.3024018704891205, -0.6..."
3,6/6/2021 16:12:46,6/6/2021 16:16:16,213.127.76.145,1,6,UgwUPFScjJ0MCeaP2F54AaABAg.8lvp3fc9Euf8lvvgsUgEgV,769,0,0,1,...,NaN,NaN,NaN,NaN,NaN,0,2018,2018.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.27884048223495483, -0.3008716404438019, -0...."
4,6/13/2021 13:25:49,6/13/2021 13:27:28,213.127.82.232,1,6,UgwWKCWtSJdFvjGHvTp4AaABAg.8kUC5dGrQ2H8kUDRihE2f3,1206,0,0,1,...,NaN,NaN,NaN,NaN,NaN,0,2018,2018.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.5210463404655457, 0.03200243413448334, -0.3..."


In [6]:
grouped_data = dataset.groupby("Platform")
dist = torch.nn.PairwiseDistance()
resultsplatform: typing.Dict[typing.Tuple[str, str], float] = {}
for model_1, c_1 in tqdm.tqdm(grouped_data['embed_MXBAI'], total=grouped_data.ngroups):
    for model_2, c_2 in tqdm.tqdm(grouped_data['embed_MXBAI'], total=grouped_data.ngroups):

        if (
            (model_1, model_2) in resultsplatform.keys() or 
            (model_2, model_1) in resultsplatform.keys()
        ):
            continue

        res = sum([
            sum(dist(
                torch.tensor(np.array(v_1)), 
                torch.tensor(np.array(c_2.tolist()))
                )) / len(c_2)
            for v_1 in c_1
        ]) / len(c_1)

        resultsplatform[(model_1, model_2)] = res

        print(f'{model_1}:{model_2}:{res.item()}')

 50%|█████     | 1/2 [01:15<01:15, 75.71s/it]

1:1:8.274992962778118



 50%|█████     | 1/2 [01:30<01:30, 90.35s/it]

1:2:7.51343754601772



100%|██████████| 2/2 [01:33<00:00, 46.90s/it]

2:2:6.060459538546106


In [7]:
for group in ["topiccode", "Genre"]:
    grouped_data = dataset.groupby(group)
    dist = torch.nn.PairwiseDistance()
    resultsplatform: typing.Dict[typing.Tuple[str, str], float] = {}
    for model_1, c_1 in tqdm.tqdm(grouped_data['embed_MXBAI'], total=grouped_data.ngroups):
        for model_2, c_2 in tqdm.tqdm(grouped_data['embed_MXBAI'], total=grouped_data.ngroups):
    
            if (
                (model_1, model_2) in resultsplatform.keys() or 
                (model_2, model_1) in resultsplatform.keys()
            ):
                continue
    
            res = sum([
                sum(dist(
                    torch.tensor(np.array(v_1)), 
                    torch.tensor(np.array(c_2.tolist()))
                    )) / len(c_2)
                for v_1 in c_1
            ]) / len(c_1)
    
            resultsplatform[(model_1, model_2)] = res
    
            print(f'Distance between {group} {model_1}:{model_2}:{res.item()}')

 20%|██        | 1/5 [00:02<00:11,  2.75s/it]

Distance between topiccode 0:0:7.761249318601641



 40%|████      | 2/5 [00:06<00:09,  3.26s/it]

Distance between topiccode 0:1:8.075305272859095



 60%|██████    | 3/5 [00:09<00:06,  3.43s/it]

...

Distance between Genre 1:1:7.763014220220769



 67%|██████▋   | 2/3 [00:53<00:26, 26.03s/it]

Distance between Genre 1:2:7.975648076236652



100%|██████████| 3/3 [01:04<00:00, 21.47s/it]

Distance between Genre 2:2:8.054402602772836


In [ ]:
#create BERT-base embeddings for each word:

model_name = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [ ]:
#create Twhin-BERT-base embeddings for each word:
model_name = "Twitter/twhin-bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [ ]:
#calculate DiffCSE embeddings:
model_name = "voidism/diffcse-roberta-base-sts"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [13]:
#function for encoding sentence embeddings based on the DiffCSE approach which is based on (and refers to) SimCSE/evaluation.py 
def encode_sentence(sentence):
    input_ids = tokenizer.encode(sentence, add_special_tokens=True, truncation=True, max_length=512, padding='max_length')
    input_ids_tensor = torch.tensor(input_ids).unsqueeze(0)  # Add batch dimension
    attention_mask = torch.ones_like(input_ids_tensor)  # Creating attention mask 
    with torch.no_grad():
            outputs = model(input_ids_tensor, output_hidden_states=True, return_dict=True, attention_mask=attention_mask)
            sentence_embedding = outputs.last_hidden_state[:, 0].cpu()
            return sentence_embedding.tolist()

In [ ]:
#for DiffCSE: Create embeddings for each argument and store them in a new column
dataset.loc[:, 'DiffCSE_embedding'] = dataset["commentText"].apply(encode_sentence)